# Add Plots Demo

This notebook demonstrates how to programmatically add new plots to an existing dashboard using the `nanohubdashboard` library.

In [ ]:
import sys
import os
import copy

# Add project root to path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../')))

from nanohubdashboard.dashboard import Dashboard
from nanohubdashboard.graph import Graph
from nanohubdashboard.plot import Plot
from nanohubremote.session import Session

## 1. Setup Session and Load Dashboard

Initialize session and load the target dashboard.

In [ ]:
print("1. Creating session...")
session = Session()

print("2. Loading dashboard...")
dashboard = Dashboard(session)
dashboard.load(8)
print(f"   ✓ Loaded: {dashboard}")

## 2. Identify Target Graph

Find a graph to use as a template or to modify. Here we look for a graph in 'COLUMN1' with 'bar' plots.

In [ ]:
target_graph = None
for graph in dashboard.graphs:
    if graph.zone == 'COLUMN1' and any(p.type == 'bar' for p in graph.plots):
        target_graph = graph
        break

if not target_graph:
    print("Could not find suitable graph to clone")
    sys.exit(1)

print(f"   Found target graph: {target_graph.id} with {len(target_graph.plots)} plots")

## 3. Clone and Modify Graph

Create a new graph based on the target graph, but modify its properties.

In [ ]:
print("\n3. Creating new graph with additional plots...")

# Create a new graph config based on the existing one
new_graph_config = copy.deepcopy(target_graph)
new_graph_config.id = "_new_graph_1"
new_graph_config.zone = "COLUMN2"  # Move to different zone
new_graph_config.priority = 0      # Put at top

# Get the base plot config to clone
base_plot_config = target_graph.plots[0].config

# Create 3 new plots with different types
new_plots = []
plot_types = ['scatter', 'line', 'bar']

for i, ptype in enumerate(plot_types):
    # Clone config
    p_config = copy.deepcopy(base_plot_config)
    p_config['type'] = ptype
    p_config['name'] = f"New {ptype.title()}"
    
    # Create Plot object
    new_plot = Plot(p_config, i)
    new_plots.append(new_plot)

new_graph_config.plots = new_plots
new_graph_config.plot_config = [p.config for p in new_plots]

# Add to dashboard
dashboard.graphs.append(new_graph_config)
print(f"   ✓ Added new graph with {len(new_plots)} plots: {plot_types}")

## 4. Visualize Result

Generate the HTML preview to see the new graph.

In [ ]:
print("\n4. Generating visualization...")
html_path = "dashboard_add_plots.html"
dashboard.visualize(filename=html_path, open_browser=True)

print(f"\n✓ DONE! Dashboard saved to: {html_path}")